In [25]:
import pandas as pd
from konlpy.tag import Twitter

In [28]:
naver_user = pd.read_csv('[Revised_c]Final_Naver_user_table.csv',sep='\t',encoding='utf-8')

In [21]:
dbdbdeep_user = pd.read_csv('[Revised]Final_dbdbdeep_user_table.csv',sep='\t',encoding='utf-8')

In [22]:
dbdbdeep_user['Categories'] = dbdbdeep_user['Categories'].apply(lambda x : refined_category(x))

In [23]:
dbdbdeep_user['Categories'].iloc[0]

[(24, "'소소한 일상'"),
 (6, "'photo'"),
 (0, "'꼼지락꼼지락♬'"),
 (0, "'영화이야기♪'"),
 (1, "'맛집냠냠◁'")]

In [24]:
naver_user.to_csv('[Revised]Final_Naver_user_table.csv',sep='\t',encoding='utf-8',index=False)
dbdbdeep_user.to_csv('[Revised]Final_dbdbdeep_user_table.csv',sep='\t',encoding='utf-8',index=False)

In [9]:
def refined_category(category):
    refined_list = []
    for i in category.split('], '):
        try:
            category_count = int(i.split(', ')[0].replace('[','').replace("'",''))
            categoty_title = i.split(', ')[1]
            categoty_title = i.split(', ')[1].replace(']]','')
            categoty_title = categoty_title.replace(')]','')
            refined_list.append((category_count,categoty_title))
        except:
            pass
    return refined_list

In [4]:
def refined_category_title(category):
    refined_list = []
    for i in category.split('], '):
        try:
            categoty_title = i.split(', ')[1]
            categoty_title = i.split(', ')[1].replace(']]','')
            categoty_title = categoty_title.replace(')]','')
            refined_list.append((categoty_title))
        except:
            pass
    return refined_list

In [5]:
korean_stopword = pd.read_csv('Korean_stopwords.txt',encoding='utf-8',header=None)
korean_stopword_list = korean_stopword[0].tolist()
stopwords = ['하다', ',', '들', '이', '..', '.', '것', '다', '이다', '~', '그', '그녀', '저', '...', '"', '~~',"'",':','&','[',']','{','}','(',')','-'] 
sum_stopwords = stopwords + korean_stopword_list

In [6]:
def Korean_tagger(Text):

    twitter = Twitter()

    try:
        
        
        malist = twitter.pos(Text, norm=True, stem=True)
        r = []
        remove_tag =['Eomi','PreEomi','Josa','Determiner','Foreign','Alpha','Number','Punctuation','Suffix','Unknown','Hashtag','KoreanParticle','ScreenName']

        stopwords = sum_stopwords


        for word in malist:
            # 어미/조사/구두점/ㅋㅋ^^ㅎㅎ/음표살림/Alphabet/부사는 대상에서 제외
            if not word[1] in remove_tag:
                if not word[0] in r:
                    if not word[0] in stopwords:
                       # 숫자, 특수문자 제거.
                        r.append(word[0])
        return ' '.join(r)
    except Exception as e:
        return ''

In [7]:
naver_user['Categories_embedding'] = naver_user['Categories'].apply(lambda x : "".join(refined_category_title(x)))
naver_user['Categories_embedding'] = naver_user['Categories_embedding'].apply(lambda x : Korean_tagger(x))

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [8]:
dbdbdeep_user['Categories_embedding'] = dbdbdeep_user['Categories'].apply(lambda x : "".join(refined_category_title(x)))
dbdbdeep_user['Categories_embedding'] = dbdbdeep_user['Categories_embedding'].apply(lambda x : Korean_tagger(x))

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [9]:
Total_user = pd.concat([naver_user,dbdbdeep_user])

In [10]:
Total_test = Total_user[['Categories_embedding','Credibility']]

In [11]:
Categories_text = Total_test['Categories_embedding'].tolist()

## Tfidf

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(max_features=391)
x = v.fit_transform(Categories_text)

In [13]:
y = Total_test['Credibility'].tolist()

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x , y , test_size=0.33, random_state=42)

In [15]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

result = LinearSVC(random_state=0).fit(x_train, y_train)
result.predict(x_test)
print("SVM :",accuracy_score(result.predict(x_test),y_test))

SVM : 0.8300395256916996


In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test, result.predict(x_test)))

             precision    recall  f1-score   support

          0       0.85      0.91      0.88       672
          1       0.79      0.67      0.73       340

avg / total       0.83      0.83      0.83      1012



In [17]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(result, x, y, cv=5)
scores                                              

array([0.85504886, 0.84201954, 0.72430669, 0.8120915 , 0.81372549])

## Doc2vec

In [18]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [19]:
Total_test = Total_user[['Categories_embedding','Credibility']]

In [20]:
text = Total_test['Categories_embedding'].tolist()

In [21]:
sentences = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(text)]

In [22]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [23]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [24]:
doc_vectorizer = Doc2Vec(
    dm=0,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=8,        # distance between the predicted word and context words
    size=391,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=20,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    workers=cores,   # multi cpu
    hs = 1,          # hierarchical softmax / default 0
    negative = 10,   # negative sampling / default 5
)

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2018-09-04 12:37:15,424 : WARNING : consider setting layer size to a multiple of 4 for greater performance


In [25]:
doc_vectorizer.build_vocab(sentences)

2018-09-04 12:37:15,432 : INFO : collecting all words and their counts
2018-09-04 12:37:15,433 : WARNING : Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.
2018-09-04 12:37:15,434 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-09-04 12:37:15,472 : INFO : collected 1337 word types and 3065 unique tags from a corpus of 3065 examples and 229662 words
2018-09-04 12:37:15,473 : INFO : Loading a fresh vocabulary
2018-09-04 12:37:15,474 : INFO : min_count=20 retains 613 unique words (45% of original 1337, drops 724)
2018-09-04 12:37:15,476 : INFO : min_count=20 leaves 225765 word corpus (98% of original 229662, drops 3897)
2018-09-04 12:37:15,478 : INFO : deleting the raw counts dictionary of 1337 items
2018-09-04 12:37:15,478 : INFO : sample=0.001 downsamples 78 most-common words
2018-09-04 12:37:15,479 : INFO : downsampling leaves estimated 126697 word corpus (56.1% of prior 225765)
201

In [26]:
print(str(doc_vectorizer))

Doc2Vec(dbow+w,d391,n10,hs,w8,mc20,s0.001,t12)


In [27]:
doc_vectorizer.corpus_count*0.03

91.95

In [28]:
doc_vectorizer.iter

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


5

In [29]:
import time
start = time.time()
for epoch in range(10):
    doc_vectorizer.train(sentences, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay
end = time.time()
print("During Time: {}".format(end-start))

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  after removing the cwd from sys.path.
2018-09-04 12:37:15,610 : INFO : training model with 12 workers on 613 vocabulary and 391 features, using sg=1 hs=1 sample=0.001 negative=10 window=8
2018-09-04 12:37:16,505 : INFO : worker thread finished; awaiting finish of 11 more threads
2018-09-04 12:37:16,776 : INFO : EPOCH 1 - PROGRESS: at 37.88% examples, 64741 words/s, in_qsize 10, out_qsize 1
2018-09-04 12:37:16,778 : INFO : worker thread finished; awaiting finish of 10 more threads
2018-09-04 12:37:16,781 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-09-04 12:37:16,803 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-09-04 12:37:16,803 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-04 12:37:16,813 : INFO : worker thread finish

2018-09-04 12:37:23,066 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-04 12:37:23,072 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-04 12:37:23,082 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-04 12:37:23,088 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-04 12:37:23,088 : INFO : EPOCH - 1 : training on 229662 raw words (129577 effective words) took 1.2s, 104773 effective words/s
2018-09-04 12:37:23,945 : INFO : worker thread finished; awaiting finish of 11 more threads
2018-09-04 12:37:24,271 : INFO : EPOCH 2 - PROGRESS: at 38.60% examples, 62614 words/s, in_qsize 9, out_qsize 3
2018-09-04 12:37:24,272 : INFO : worker thread finished; awaiting finish of 10 more threads
2018-09-04 12:37:24,274 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-09-04 12:37:24,275 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-09-04 12:37:24,276

2018-09-04 12:37:30,634 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-04 12:37:30,646 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-04 12:37:30,647 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-04 12:37:30,647 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-04 12:37:30,648 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-04 12:37:30,648 : INFO : EPOCH - 2 : training on 229662 raw words (129688 effective words) took 1.2s, 105039 effective words/s
2018-09-04 12:37:31,521 : INFO : worker thread finished; awaiting finish of 11 more threads
2018-09-04 12:37:31,809 : INFO : EPOCH 3 - PROGRESS: at 38.60% examples, 63888 words/s, in_qsize 10, out_qsize 1
2018-09-04 12:37:31,810 : INFO : worker thread finished; awaiting finish of 10 more threads
2018-09-04 12:37:31,815 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-09-04 12:37:31,83

2018-09-04 12:37:38,332 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-04 12:37:38,359 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-04 12:37:38,387 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-04 12:37:38,389 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-04 12:37:38,397 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-04 12:37:38,403 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-04 12:37:38,404 : INFO : EPOCH - 3 : training on 229662 raw words (129737 effective words) took 1.4s, 94761 effective words/s
2018-09-04 12:37:39,395 : INFO : worker thread finished; awaiting finish of 11 more threads
2018-09-04 12:37:39,687 : INFO : EPOCH 4 - PROGRESS: at 38.60% examples, 57925 words/s, in_qsize 10, out_qsize 1
2018-09-04 12:37:39,689 : INFO : worker thread finished; awaiting finish of 10 more threads
2018-09-04 12:37:39,714

2018-09-04 12:37:46,748 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-04 12:37:46,753 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-04 12:37:46,757 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-04 12:37:46,758 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-04 12:37:46,766 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-04 12:37:46,772 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-04 12:37:46,786 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-04 12:37:46,787 : INFO : EPOCH - 4 : training on 229662 raw words (129680 effective words) took 1.4s, 94134 effective words/s
2018-09-04 12:37:47,782 : INFO : worker thread finished; awaiting finish of 11 more threads
2018-09-04 12:37:48,053 : INFO : EPOCH 5 - PROGRESS: at 38.04% examples, 58767 words/s, in_qsize 10, out_qsize 1
2018-09-04 12:37:48,054 

2018-09-04 12:37:55,052 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-04 12:37:55,053 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-04 12:37:55,065 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-04 12:37:55,068 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-04 12:37:55,085 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-04 12:37:55,088 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-04 12:37:55,089 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-04 12:37:55,095 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-04 12:37:55,095 : INFO : EPOCH - 5 : training on 229662 raw words (129879 effective words) took 1.3s, 96434 effective words/s
2018-09-04 12:37:55,096 : INFO : training on a 1148310 raw words (648247 effective words) took 6.9s, 93380 effective words/s
2018-09-04 12

2018-09-04 12:38:02,897 : INFO : worker thread finished; awaiting finish of 11 more threads
2018-09-04 12:38:03,103 : INFO : worker thread finished; awaiting finish of 10 more threads
2018-09-04 12:38:03,115 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-09-04 12:38:03,123 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-09-04 12:38:03,132 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-04 12:38:03,153 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-04 12:38:03,158 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-04 12:38:03,161 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-04 12:38:03,168 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-04 12:38:03,171 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-04 12:38:03,177 : INFO : worker thread finished; awaiting finish of 1 more threa

2018-09-04 12:38:10,807 : INFO : worker thread finished; awaiting finish of 11 more threads
2018-09-04 12:38:11,150 : INFO : EPOCH 2 - PROGRESS: at 38.60% examples, 61158 words/s, in_qsize 10, out_qsize 1
2018-09-04 12:38:11,175 : INFO : worker thread finished; awaiting finish of 10 more threads
2018-09-04 12:38:11,176 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-09-04 12:38:11,178 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-09-04 12:38:11,181 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-04 12:38:11,185 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-04 12:38:11,191 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-04 12:38:11,197 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-04 12:38:11,203 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-04 12:38:11,231 : INFO : worker thread finished; awaiting 

2018-09-04 12:38:18,586 : INFO : worker thread finished; awaiting finish of 11 more threads
2018-09-04 12:38:18,914 : INFO : EPOCH 3 - PROGRESS: at 38.60% examples, 61654 words/s, in_qsize 10, out_qsize 1
2018-09-04 12:38:18,918 : INFO : worker thread finished; awaiting finish of 10 more threads
2018-09-04 12:38:18,957 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-09-04 12:38:18,957 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-09-04 12:38:18,958 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-04 12:38:18,962 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-04 12:38:18,963 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-04 12:38:18,965 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-04 12:38:18,967 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-04 12:38:18,980 : INFO : worker thread finished; awaiting 

During Time: 65.95884346961975


In [30]:
X_train = [doc_vectorizer.infer_vector(doc.words) for doc in sentences]

In [31]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train , y , test_size=0.33, random_state=42)

In [32]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

result = LinearSVC(random_state=0).fit(x_train, y_train)
result.predict(x_test)
print("SVM :",accuracy_score(result.predict(x_test),y_test))

SVM : 0.7796442687747036


In [33]:
from sklearn.neural_network import MLPClassifier

In [34]:
mlp_clf = MLPClassifier(
    hidden_layer_sizes=(50,),
    max_iter=10,
    alpha=1e-4,
    solver='sgd',
    verbose=10,
    tol=1e-4,
    random_state=1,
    learning_rate_init=.1
)

In [38]:
start = time.time()
mlp_clf.fit(x_train, y_train)
end = time.time()
print('Time: {:f}s'.format(end-start))

Iteration 1, loss = 0.62487071
Iteration 2, loss = 0.47994807
Iteration 3, loss = 0.36777816
Iteration 4, loss = 0.31784093
Iteration 5, loss = 0.29063482
Iteration 6, loss = 0.26936735
Iteration 7, loss = 0.25290561
Iteration 8, loss = 0.23733652
Iteration 9, loss = 0.22322287
Iteration 10, loss = 0.20955831
Time: 0.139953s


C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [39]:
y_pred = mlp_clf.predict(x_test)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_pred, y_test)))

테스트 정확도: 0.826
